<a href="https://colab.research.google.com/github/SashankRaja/BERTsentiment/blob/main/BERTsentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importing the dependencies
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in indexes: https://download.pytorch.org/whl/cu124


In [3]:
#let us now import the dependencies we need
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch #extracts highest seq result
import requests #Used to grab webpage
from bs4 import BeautifulSoup #allows us to traverse through what we get from webpage through requests
import re
#AutoTokenizer allows us to convert the string into numeric values to pass into the transformer
#AutoModelForSequenceClassification gives us the architecture from transformer to load in the NLP model

In [4]:
#lets instantiate the model
#For this project we are using a pretrained model from hugging face, so lets load that in
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [11]:
#Lets now check out the model
tokens = tokenizer.encode('Man it was horrible,I HATED it',return_tensors='pt')
tokens
#return_tensors='pt' we do pt as we are using pytorch so it returns tensors which is appropriate to pytorch
#tensor= raw data (usually text) is first converted into numeric representations (tokens) through a tokenizer, which is then converted into tensors for the model to process.
#basically the input which we give into the transformer should be a tensor

tensor([[  101, 10564, 10197, 10140, 36129, 45795, 10301,   117,   151, 39487,
         10163, 10197,   102]])

In [13]:
#we can even decode the above to see the original sentense
tokendc=tokenizer.decode(tokens[0])
tokendc

'[CLS] man it was horrible, i hated it [SEP]'

In [17]:
#let us now pass the token to our model
result=model(tokens)
result
#as we can see the result returs us a tensor of 5 values this is telling us how mcuh preference the model is giving us for that class
#we can see that it goes like 4.2,2,etc etc the higher the value the more the pref to assign that class as then sentiment since class 0(1 star) has the higehst value that means
#the model really wants to give it a 1 star rating.
#The model has not returned the hidden states in this output, which means you did not request the intermediate layer outputs.
# attentions=None:Similarly, attention weights have not been returned here. Attention weights represent how much attention the model pays to each token in the input
#sequence when making predictions for each position.

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.2608,  2.0110, -0.1768, -2.6033, -2.6546]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [28]:
result.logits
#we need the max value from this as that is the rating the model predicts thus we do
int(torch.argmax(result.logits)+1)#we do +1 since the transformer goes from 0 to 4 but we need a star rating from 1 to 5
#we use argmax since we want which class(or index) has the most probability and we dont want the actual max value

1

In [147]:
#Let us now collect the reviews using data scraping
import requests
from bs4 import BeautifulSoup
# Make a request to the webpage
r = requests.get('https://myanimelist.net/anime/21/One_Piece/reviews')
# Parse the HTML content
soup = BeautifulSoup(r.text, 'html.parser')
# Find all review elements using the class name
reviews = soup.find_all('div', class_='review-element js-review-element')
# Extract the review text from each element
review_texts = []
for review in reviews:
  text_element = review.find('div', class_='text')
  if text_element:
    review_texts.append(text_element.text.strip())
# Print the extracted reviews
print(review_texts)

['There are two responses I get, without fail, every time I try to get someone new to watch this show.  "I don\'t like the art style" or "I\'ve seen the dub - NO THANKS."  I\'m guilty of both of these myself.  But if there\'s one thing I need to stress before even getting started on this review, it\'s that the 4Kids dub is NOT One Piece.  For the love of god, PLEASE do not think it is.  If you\'ve suffered the misfortune of seeing some of the 4Kids episodes, just erase them from your mind and start fresh.  They\n                  ...\nbutchered it, there\'s really no other way to put it.  They cut episodes, changed the dialogue to fit a MUCH younger and apparently far less intelligent audience (almost insultingly so), gave the characters RIDICULOUS voices, and pretty much watered down the entire series.  FUNimation has done a much better job so far from what I\'ve seen, but regardless, watching it in its original Japanese form with subtitles is really the way to go.\n\r\nAs for the art

In [158]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer

def lemmatize_reviews(review_texts):
  lemmatizer = WordNetLemmatizer()
  stop_words = set(stopwords.words('english'))  # Load English stop words

  lemmatized_texts = []
  for text in review_texts:
    words = text.split()
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    lemmatized_texts.append(' '.join(filtered_words))

  return lemmatized_texts
lemmatized_reviews = lemmatize_reviews(review_texts)
#we peroform lemmatamization and removal of all stop words from the reviews in hopes of allowing the transformer to give a more accurate prediction of the sentiment
#Another reason to do this is in order to shorten the amount of tokens that our transformer needs to process and hence work in an more optimised manner(max token for hugging face
#is 512 so we do all this in hopes to ensure only the important words or words carrying more weights are tokenized)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [159]:
#let us now load the reviews into a dataframe and score it
import numpy as np
import pandas as pd
df = pd.DataFrame(np.array(lemmatized_reviews), columns=['review'])

In [160]:
df.tail()
#checking last 5 reviews(wkt we have a total of n reviews to work with now)

,review
15,"Yes, believe eyes, I really rated One Piece 6...."
16,One Piece honestly amazing series I ever watch...
17,''I'm Luffy! The Man Who Will Become The Pirat...
18,You know I really miss? The good ol' day shone...
19,One Piece definitely must-watch anime series a...


In [161]:
#now that we have all our reviews neatly in a dataset we need to create a fucntion where we pass each review into the model so it can generate the score for it
def sentiment_score(lemmatized_reviews):
    tokens = tokenizer.encode(lemmatized_reviews,truncation=True,truncation_strategy="only_last",max_length=512,return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1
#We use truncation true as the max tokens we can pass into the hugging face model is 512 and hence every other token after that is deleted but here our trunkation
#strat is to include only the last I.e only last 512 tokens are considered if more than 512 tokens are there.I have chosen this since usually people conclude their
#reviews in the last part of the paragraphs and that is where the more imp tokens would be concentrated.

In [162]:
sentiment_score(df['review'].iloc[5])

4

In [163]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[-512:]))
#we make a new coulmn names sentiment where we basically loop through each review using the lambda function and basically pass each of this review into our function
#and then the assigned score to that review is saved into the sentiment coulmn.

In [164]:
df

,review,sentiment
0,"There two response I get, without fail, every ...",5
1,Warning: Minor Spoilers If I say anything bad ...,2
2,One Piece far best shounen anime I watched. Bu...,2
3,"Review In short, I'm Going keep real would obv...",2
4,Lets start story. Story. 2/10. The story boy n...,1
5,"In personal opinion, One Piece one anime easy ...",4
6,"OnePiIISSWuaaaa!!! From scale ""over 9000"" ""too...",2
7,"One Piece recount story Monkey D Luffy, young ...",1
8,I written review almost year might well start ...,5
9,Ask person internet favourite kind warrior bou...,3


In [165]:
su=0
for i in df['sentiment']:
  su+=i
print(su/19)

3.210526315789474
